In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import pathlib
import os

In [2]:
file_location = 'E:/QGIS/gkpg'
population = '/berlin_population.gpkg'
district = '/berlin_districts.gpkg'
cycle = '/cycle.gpkg'

In [3]:
population = gpd.read_file(file_location + population)
district = gpd.read_file(file_location + district)
cycle = gpd.read_file(file_location + cycle)

In [4]:
population = population.fillna(0)


In [5]:
population.columns

Index(['schluessel', 'ew2022', 'flalle', 'ew_ha_2022', 'alter_u6',
       'alter_6_u10', 'alter_10_u18', 'alter_18_u65', 'alter_65_u70',
       'alter_70_u75', 'alter75_u80', 'alter_80plus', 'typklar', 'geometry'],
      dtype='object')

In [6]:
population = population[['schluessel','ew2022','geometry']]

In [7]:
population.head()

,schluessel,ew2022,geometry
0,0100980011000100,0,"MULTIPOLYGON (((389424.849 5821759.519, 389373..."
1,0100980011000200,0,"MULTIPOLYGON (((389424.849 5821759.519, 389336..."
2,0100980011000300,18,"MULTIPOLYGON (((389326.319 5821907.843, 389287..."
3,0100980021000200,850,"MULTIPOLYGON (((389743.255 5822026.125, 389703..."
4,0100980021000300,3,"MULTIPOLYGON (((389693.578 5821875.268, 389605..."


In [8]:
cycle.head()

,gisid,elem_nr,ist_radvorrangnetz,existenz,geometry
0,1,37430014_37430015.02,Radvorrangnetz,"ja, in Straßennetz","MULTILINESTRING ((382122.937 5808289.889, 3821..."
1,2,54600040_54600027.02,Radvorrangnetz,"ja, in Straßennetz","MULTILINESTRING ((399564.997 5825717.847, 3995..."
2,3,49470035_49470036.02,Radvorrangnetz,"ja, in Straßennetz","MULTILINESTRING ((394182.419 5812758.101, 3942..."
3,4,52520034_53520013.02,Radvorrangnetz,"ja, in Straßennetz","MULTILINESTRING ((397751.848 5817229.197, 3977..."
4,5,44570004_44570013.01,Radvorrangnetz,"ja, in Straßennetz","MULTILINESTRING ((389353.057 5822475.861, 3893..."


In [9]:
cycle = cycle[['gisid','geometry']]

In [10]:
population = population.to_crs(3857)

In [11]:
# Ensure both datasets are in the same CRS
cycle = cycle.to_crs(population.crs)
district = district.to_crs(population.crs)

In [12]:
print(cycle.crs)
print(district.crs)

EPSG:3857
EPSG:3857


In [13]:
# 1. Calculate district population density
district['area in sqkm'] = district.geometry.area / 1000000  # area in sq km
district_area = district

In [14]:
# Perform spatial join
# op='intersects' means it will join if the geometries intersect
# how='left' keeps all districts and adds population data where available
district = gpd.sjoin(district, population, how='inner', predicate='contains')



In [15]:
district.tail()

,district,geometry,area in sqkm,index_right,schluessel,ew2022
11,Reinickendorf,"MULTIPOLYGON (((1471433.794 6908101.768, 14715...",241.658472,22656,2000926051000700,0
11,Reinickendorf,"MULTIPOLYGON (((1471433.794 6908101.768, 14715...",241.658472,22654,2000926051000500,0
11,Reinickendorf,"MULTIPOLYGON (((1471433.794 6908101.768, 14715...",241.658472,22651,2000926051000200,0
11,Reinickendorf,"MULTIPOLYGON (((1471433.794 6908101.768, 14715...",241.658472,22415,2000920021000100,0
11,Reinickendorf,"MULTIPOLYGON (((1471433.794 6908101.768, 14715...",241.658472,22411,2000920011000100,124


In [16]:
district = district[['district','geometry','area in sqkm', 'ew2022']]
district.head()

,district,geometry,area in sqkm,ew2022
0,Mitte,"MULTIPOLYGON (((1483591.158 6898621.376, 14835...",106.299293,0
0,Mitte,"MULTIPOLYGON (((1483591.158 6898621.376, 14835...",106.299293,0
0,Mitte,"MULTIPOLYGON (((1483591.158 6898621.376, 14835...",106.299293,519
0,Mitte,"MULTIPOLYGON (((1483591.158 6898621.376, 14835...",106.299293,391
0,Mitte,"MULTIPOLYGON (((1483591.158 6898621.376, 14835...",106.299293,879


In [ ]:
# If multiple population points fall within a district, you might want to aggregate
# For example, to sum the population:
district = district.dissolve(by='district', aggfunc='sum')

In [ ]:
district.head()

In [ ]:
# Create a output path for the data
# output_fp = "E:/QGIS/gkpg/berlin_district_population.gpkg"

# Write the file
# district.to_file(output_fp)

In [ ]:
# 2. Analyze cycling network coverage
# Create a 500m buffer around the cycling network
cycling_buffer = cycle.buffer(500)
cycling_buffer = gpd.GeoDataFrame(geometry=gpd.GeoSeries(cycling_buffer.unary_union))
cycling_buffer.crs = cycle.crs

In [ ]:
cycling_buffer.plot()

In [ ]:
# Calculate the intersection of buffer with districts
district['cycling_area'] = district.intersection(cycling_buffer.unary_union).area
district['cycling_coverage'] = district['cycling_area'] / district.geometry.area

In [ ]:
district.head()

In [ ]:
# district = district.merge(district_area, on='district', how='left')